<a href="https://colab.research.google.com/github/mkbahk/AmazonBraket/blob/main/QuantumApplicationAlgorithm_VQA_QAOA(GoogleORTool)_mkbahk_20260203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.8/29.8 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.33.5 which is incompatible.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5

In [4]:
import numpy as np
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

In [13]:
def create_data_model():
    """문제 데이터를 정의합니다."""
    data = {}
    # 100개의 무작위 도시 좌표 (0~100 사이)
    np.random.seed(42)
    locations = np.random.randint(0, 200, size=(200, 2))

    # 거리 행렬 생성 (Euclidean distance)
    num_locations = len(locations)
    dist_matrix = np.zeros((num_locations, num_locations))
    for i in range(num_locations):
        for j in range(num_locations):
            dist_matrix[i][j] = np.hypot(locations[i][0] - locations[j][0],
                                         locations[i][1] - locations[j][1])
        ###for
    ###for

    data["distance_matrix"] = dist_matrix.astype(int).tolist()
    data["num_vehicles"] = 1 # 외판원은 1명
    data["depot"] = 0        # 시작점 (0번 도시)
    return data, locations
###def


In [14]:
def solve_tsp():
    # 1. 데이터 준비
    data, locations = create_data_model()

    # 2. Routing Index Manager 및 Model 설정
    manager = pywrapcp.RoutingIndexManager(len(data["distance_matrix"]),
                                           data["num_vehicles"], data["depot"])
    routing = pywrapcp.RoutingModel(manager)

    # 3. 거리 계산 콜백 함수 등록
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]
    ###def

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # 4. 여행 비용(거리) 설정
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # 5. 검색 파라미터 설정 (Heuristic 사용)
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # 6. 문제 해결
    solution = routing.SolveWithParameters(search_parameters)

    # 7. 결과 출력
    if solution:
        print_solution(manager, routing, solution)
    ###if
###def

In [15]:
def print_solution(manager, routing, solution):
    print(f"최적 경로 탐색 완료!")
    index = routing.Start(0)
    plan_output = "경로: "
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += f"{manager.IndexToNode(index)} -> "
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    ###while

    plan_output += f"{manager.IndexToNode(index)}\n"
    print(plan_output)
    print(f"총 이동 거리: {route_distance} units")
###def

In [16]:
if __name__ == "__main__":
    solve_tsp()
###if

최적 경로 탐색 완료!
경로: 0 -> 67 -> 194 -> 148 -> 73 -> 142 -> 114 -> 31 -> 176 -> 44 -> 12 -> 18 -> 116 -> 167 -> 19 -> 41 -> 125 -> 77 -> 56 -> 76 -> 8 -> 146 -> 111 -> 117 -> 112 -> 164 -> 121 -> 128 -> 126 -> 98 -> 181 -> 135 -> 189 -> 159 -> 151 -> 95 -> 165 -> 84 -> 90 -> 198 -> 48 -> 92 -> 188 -> 108 -> 101 -> 131 -> 199 -> 163 -> 149 -> 152 -> 65 -> 100 -> 138 -> 89 -> 29 -> 55 -> 60 -> 54 -> 96 -> 63 -> 115 -> 190 -> 160 -> 107 -> 88 -> 123 -> 97 -> 33 -> 3 -> 17 -> 32 -> 57 -> 136 -> 147 -> 145 -> 40 -> 37 -> 22 -> 141 -> 184 -> 166 -> 91 -> 191 -> 23 -> 71 -> 178 -> 36 -> 68 -> 58 -> 93 -> 140 -> 38 -> 137 -> 70 -> 86 -> 46 -> 139 -> 2 -> 26 -> 28 -> 15 -> 72 -> 87 -> 81 -> 1 -> 124 -> 52 -> 9 -> 103 -> 180 -> 43 -> 173 -> 27 -> 172 -> 50 -> 162 -> 192 -> 62 -> 78 -> 85 -> 118 -> 59 -> 14 -> 177 -> 193 -> 21 -> 83 -> 132 -> 45 -> 64 -> 174 -> 185 -> 42 -> 144 -> 183 -> 109 -> 69 -> 79 -> 61 -> 134 -> 66 -> 179 -> 196 -> 13 -> 169 -> 47 -> 161 -> 104 -> 11 -> 25 -> 195 -> 175 -> 170 